In [1]:
import os
from SimpleITK.SimpleITK import And
import numpy as np
import scipy
import SimpleITK as sitk
import matplotlib.pyplot as plt
from skimage.segmentation import join_segmentations
import pandas as pd
from utils import StapleGun4, ImageFlipper


In [2]:
def GetSlices(seg_arr):
    values= []
    for i in range(0,len(seg_arr)):
        val = np.sum(seg_arr[i,:,:])
        #if val >=   :
        if val >= 50750:
            values.append(i)
    return values

def GetSlicesArray(masks):
    slices_array =[]
    for image in masks:
        slices = GetSlices(image)
        slices_array.append((slices))
    return np.asarray(slices_array)

def GetArea(seg_arr, staple_slices, patient):
    values = []
    for i in staple_slices[patient]:
        
        val = np.sum(seg_arr[i,:,:])
        
        val/255
        values.append(val.astype(int))
    return values
def GetAreaArray(masks, masks_ids, staple_id, staple_slices):
    area_array =[]
    masks_ids_l = masks_ids.tolist()
    for id in staple_id:
       
        place_holder = masks_ids_l.index(id)
        
        areas = GetArea(masks[place_holder], staple_slices, place_holder)
        area_array.append(areas)
    return np.asarray(area_array)


def StapleGun(masks_a, masks_b, masks_c, ids_a, ids_b, ids_c):

    """
    slices_array_a = GetSlicesArray(masks_a)
    slices_array_b = GetSlicesArray(masks_b)
    slices_array_c = GetSlicesArray(masks_c)
    """
    #removed CR slices, for when I have all of the data sheets 
    path_a = "/Users/olivia/Documents/PhD/MISTIE/mask_data/CRdata.xlsx"
 
    slices_array_a = NewRemoveCoronaRadiataSlices(masks_a, ids_a, path_a, sheetname='adrian')
    slices_array_b = NewRemoveCoronaRadiataSlices(masks_b, ids_b, path_a, sheetname = 'paul')
    slices_array_c = NewRemoveCoronaRadiataSlices(masks_c, ids_c, path_a, sheetname = 'sacha') 
    

    staple_array = []
    slices_array = []
    id = []
    id_ticker = 0
    

    for i in range (0, 77) :
        id_ticker += 1
        staple_slices_array = []  
        slices_id =[]
        patient_id = ids_a[i]

        for j in slices_array_a[i]:
            if j in slices_array_b[i] and j in slices_array_c[i]:

                masks_a[i][j][masks_a[i][j]==255] = 1
                masks_b[i][j][masks_b[i][j]==255] = 1
                masks_c[i][j][masks_c[i][j]==255] = 1

                mask_a = sitk.GetImageFromArray(masks_a[i][j])
                mask_b = sitk.GetImageFromArray(masks_b[i][j])
                mask_c = sitk.GetImageFromArray(masks_c[i][j])
                

                staple_filter = sitk.STAPLEImageFilter()
                staple_filter.SetForegroundValue(1)
                staple_image = staple_filter.Execute(mask_a,mask_b, mask_c)
                staple_image = staple_image > 0.5                                
                staple_slice_array = sitk.GetArrayFromImage(staple_image)
                iter = staple_filter.GetElapsedIterations()
                print("Patient: ", i, ", Slice: ", j)
                print("iter: ", iter)

                staple_slices_array.append(staple_slice_array)
                slices_id.append(j)
                
        id.append((patient_id))
        staple_array.append(staple_slices_array)
        slices_array.append(slices_id)
        
    return staple_array, id, slices_array

def UnpackNpz(path):
    data = np.load(path, allow_pickle=True)
    masks = data['masks']
    ids = data['ids']
    return masks, ids

def OnesNZeros(mask_array):
    mask_array_copy = mask_array
    slices_array = GetSlicesArray(mask_array_copy)
    for i in range(0, len(mask_array_copy)):
        for slic in slices_array[i]:
            mask_array_copy[i][slic][mask_array_copy[i][slic]==255] = 1
    return mask_array_copy

def HasCoronaRadiata(csv_path, sheetname):
    data_frame = pd.read_excel(csv_path, sheet_name= sheetname, usecols = ['PRIME ID ', 'CR distance to hematoma (mm)'])
    
    CR =data_frame['CR distance to hematoma (mm)'].tolist()
    ids = data_frame['PRIME ID '].tolist()
    ids = [str(x) for x in ids]
    #print("ids: ",ids)
    ind = []
    for i in range(0,len(CR)):
        if CR[i] !=0:
            ind.append(i)

    patients=[]
    for i in ind:
        #print("id data: ", ids[i])
        patient = ids[i][0:4]
        patients.append(patient)
    #print("patients: ", patients)
    return np.asarray(patients, dtype = int)

def RemoveCoronaRadiataSlices(mask_array, mask_ids, csv_path, sheetname):
    removed_slices = []
    mask_ids_dummy = mask_ids.astype(int).tolist()
    patients = HasCoronaRadiata(csv_path, sheetname)
    slices_array = GetSlicesArray(mask_array)
    for id in patients:
        if id in mask_ids_dummy:
            
            ind = mask_ids_dummy.index(id)
            length= len(slices_array[ind])
            if id != 0:
               if len(slices_array[ind]) != 0:
                  slices_array[ind].pop(length-1)
         
    return slices_array

def NewRemoveCoronaRadiataSlices(mask_array, mask_ids, csv_path, sheetname):
    
    removed_slices = []

    slices_array = GetSlicesArray(mask_array)
    mask_ids_dummy = mask_ids.astype(int).tolist()
    data_frame = pd.read_excel(csv_path, sheet_name= sheetname, usecols = ['cr missing','cr upside down' ])
    CR_upsidedown = data_frame['cr upside down'].astype(int).tolist()
    CR_missing = data_frame['cr missing'].astype(int).tolist()
    #print("ids: ",mask_ids_dummy, "upside: ",CR_upsidedown, "missing: ", CR_missing)
    
    for id in mask_ids_dummy:
        
        if id in CR_upsidedown:
            place_holder = mask_ids_dummy.index(id)
            if len(slices_array[place_holder]) != 0:
                slices_array[place_holder].pop(0)
        elif id in CR_missing:
            slices_array = slices_array
        else:
            place_holder = mask_ids_dummy.index(id)
            if len(slices_array[place_holder]) != 0:
                slices_array[place_holder].pop(len(slices_array[place_holder])-1)
    return slices_array



def IdSkip(ids_a,ids_b,ids_c, masks_a, masks_b, masks_c):
    mask_a = masks_a.tolist()
    mask_b = masks_b.tolist()
    mask_c = masks_c.tolist()
    id_a = ids_a.tolist()
    id_b = ids_b.tolist()
    id_c = ids_c.tolist()
    ids_skip =[]
    for id in ids_a:
        if id not in ids_b or id not in ids_c:
            ids_skip.append(id)
    for id in ids_skip:
        if id in ids_a:
            i = id_a.index(id)
            mask_a.pop(i)
            id_a.pop(i)

        if id in ids_b:
            i = id_b.index(id)
            mask_b.pop(i)
            id_b.pop(i)

        if id in ids_c:
            i = id_c.index(id)
            mask_c.pop(i)
            id_c.pop(i)

    return np.asarray(mask_a), np.asarray(id_a), np.asarray(mask_b), np.asarray(id_b), np.asarray(mask_c), np.asarray(id_c)

def dice_loss(input, target):
    
    smooth = 1.

    iflat = input.view(-1)
    tflat = target.view(-1)
    print("iflat: ", iflat, "tflat: ", tflat)
    intersection = (iflat * tflat).sum(-1)
    print(intersection)
    print(iflat.sum(-1) + tflat.sum(-1))
    return ((2. * intersection + smooth) /
              (iflat.sum(-1) + tflat.sum(-1) + smooth))


def compute_dice_coefficient(mask_pred, mask_gt):

  volume_sum = mask_gt.sum() + mask_pred.sum()
  if volume_sum == 0:
    return np.NaN
  volume_intersect = (mask_gt & mask_pred).sum()
  return 2*volume_intersect / volume_sum 

def DiceCaster(staple_array, staple_ids, staple_slices, masks_a, ids_a):
    
    id_a = ids_a.tolist()
    dice_array =[]
    for i in range(0, len(staple_array)):
        dices_array =[]
        for j in range(0, len(staple_array[i])):
            patient_id = staple_ids[i]
            slices_arr = staple_slices[i]
            place_holder = id_a.index(patient_id)
            ind = slices_arr[j]
            
            #print("place holder: ", place_holder, "ind: ", ind, "i: ", i, "j: ", j )
            dice_coeff = compute_dice_coefficient(masks_a[place_holder][ind,:,:], staple_array[i][j])
            dices_array.append(dice_coeff)
        dice_array.append(dices_array)   
    return dice_array

def DiceCaster2(masks_b, ids_b, slices_b, masks_a, ids_a, slices_a):
    
    id_a = ids_a.tolist()
    dice_array =[]
    for i in range(0, len(masks_b)):
        dices_array =[]
        patient_id = ids_b[i]
        print(patient_id)
        patient_id = ids_b[i]
        place_holder = id_a.index(patient_id)
        #if len(slices_b)!=  0 and len(slices_a[place_holder]) != 0:
        for j in slices_b[i]:

            print(" j: ", j, slices_b[i], slices_a[i])
            if j in slices_a[place_holder] and j <= len(masks_a[place_holder]):
                print("true")
                
                #for j in range(0, len(slices_b[i])):
                #for j in range(0, len(staple_array[i])):
        
                
                #slices_arr = slices_b[i]
                
                #ind = slices_arr[j]
                #if ind in slices_a[i]:

                #print("place holder: ", place_holder, "ind: ", ind, "i: ", i, "j: ", j )
                print("lens: ", len(masks_a[place_holder]), len(masks_b[i]))
                dice_coeff = compute_dice_coefficient(masks_a[place_holder][j,:,:], masks_b[i][j,:,:])
                dice_coeff = dice_coeff * 100
                dices_array.append(dice_coeff)
            else:
                print("false")
        dice_array.append(dices_array)   
    return dice_array
def AverageDice(dice_array):
    average = []
    for patient in dice_array:
        cleanedList = [x for x in patient if str(x) != 'nan']
        sum= np.sum(cleanedList)
        length = len(cleanedList)
        avg = sum/length
        average.append(avg)

    cleanedAverage = [x for x in average if str(x) != 'nan']
    length = len(cleanedAverage)
    sum = np.sum(cleanedAverage)
    total_avg = sum/length
    return total_avg, average

def GetSliceOverlap(slices_a, ids_a, slices_b, ids_b, slices_c, ids_c):
    percent = []
    
    total_intersection = 0
    total_union = 0
    for i in ids_a:
        ind_a = ids_a.tolist().index(i)
        ind_p= ids_b.tolist().index(i)
        ind_s = ids_c.tolist().index(i)
        print(ind_a, ind_p, ind_s)
        intersection = set(slices_a[ind_a]) & set(slices_b[ind_p]) & set(slices_c[ind_s])
        union = set(slices_a[ind_a])|set(slices_b[ind_p])|set(slices_c[ind_s])
        smooth = 0.001
        percentage = len(intersection)/(len(union)+smooth)
        percent.append(percentage)
        total_intersection += len(intersection)
        total_union += len(union)

    total_percentage = total_intersection/total_union
    return total_percentage, percent
    
def UnpackSlices(path):
    data = np.load(path, allow_pickle=True)
    slices = data['slices']
    ids = data['ids']
    return slices, ids



In [4]:
adrian_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/adrian_masks.npz'
sacha_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/sacha_masks.npz'
paul_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/paul_masks.npz'
h_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/h_masks.npz'
masks_paul, ids_paul = UnpackNpz(paul_path)
masks_adrian, ids_adrian = UnpackNpz(adrian_path)
masks_sacha, ids_sacha = UnpackNpz(sacha_path)
masks_h, ids_h = UnpackNpz(h_path)

KeyboardInterrupt: 

In [3]:
adrian_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/PLIC_mask_adrian.npz'
sacha_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/PLIC_mask_sacha.npz'
paul_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/PLIC_mask_paul.npz'
h_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/PLIC_mask_h.npz'
PLIC_masks_paul, PLIC_ids_paul = UnpackNpz(paul_path)
PLIC_masks_adrian, PLIC_ids_adrian = UnpackNpz(adrian_path)
PLIC_masks_sacha, PLIC_ids_sacha = UnpackNpz(sacha_path)
PLIC_masks_h, PLIC_ids_h = UnpackNpz(h_path)

In [ ]:
path_a = "/Users/olivia/Documents/PhD/MISTIE/mask_data/CRdata.xlsx"
adrian_slices = NewRemoveCoronaRadiataSlices(masks_adrian, ids_adrian, path_a, sheetname='adrian')
paul_slices = NewRemoveCoronaRadiataSlices(masks_paul,ids_paul, path_a, sheetname='paul')
sacha_slices = NewRemoveCoronaRadiataSlices(masks_sacha,ids_sacha, path_a, sheetname='sacha')
h_slices = NewRemoveCoronaRadiataSlices(masks_h,ids_h, path_a, sheetname='h')

In [ ]:
#print(np.unique(masks_adrian[1]))
old_slices = GetSlicesArray(masks_adrian_old)
#slices = GetSlicesArray(masks_adrian)
# oyer
# yo er   o yer   foyer

In [ ]:
ct_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CT_scan_del.npz'
ct_scans, ct_ids = UnpackNpz(ct_path)

In [ ]:

#masks_paul_01 = OnesNZeros(new_masks_paul)
areas_s = GetAreaArray(new_mask_sacha, new_id_sacha, staple_ids, staple_slices)
areas_a = GetAreaArray(new_mask_adrian, new_id_adrian, staple_ids, staple_slices)
areas_p = GetAreaArray(new_mask_paul, new_id_paul, staple_ids, staple_slices)

"""
area_df = pd.DataFrame(areas,index = staple_ids)
area_df.index.name = "Patient ID"
with pd.ExcelWriter('/Users/olivia/Documents/PhD/MISTIE/mask_data/area.xlsx') as writer:  
    area_df.to_excel(writer, sheet_name='area')
"""
flattened_s = []
area_flat_s =[]
dice_flat_s =[]
for i in range(0, len(areas_s)):
    for j in range(0, len(areas_s[i])):
        flat = (dice_array_sacha[i][j],areas_s[i][j] )
        dice = dice_array_sacha[i][j]
        area = areas_s[i][j]
        dice_flat_s.append(dice)
        area_flat_s.append(area)
        flattened_s.append(flat)

flattened_a = []
area_flat_a =[]
dice_flat_a =[]
for i in range(0, len(areas_a)):
    for j in range(0, len(areas_a[i])):
        flat = (dice_array_adrian[i][j],areas_a[i][j] )
        dice = dice_array_adrian[i][j]
        area = areas_s[i][j]
        dice_flat_a.append(dice)
        area_flat_a.append(area)
        flattened_a.append(flat)
flattened_p = []
area_flat_p =[]
dice_flat_p =[]
for i in range(0, len(areas_p)):
    for j in range(0, len(areas_p[i])):
        flat = (dice_array_paul[i][j],areas_p[i][j] )
        dice = dice_array_paul[i][j]
        area = areas_s[i][j]
        dice_flat_p.append(dice)
        area_flat_p.append(area)
        flattened_p.append(flat)

s = plt.scatter(*zip(*flattened_s))
a = plt.scatter(*zip(*flattened_a))
p = plt.scatter(*zip(*flattened_p))
plt.plot(np.unique(flattened_s[0]), np.poly1d(np.polyfit(flattened_s[0], flattened_s[1], 1))(np.unique(flattened_s[0])))
plt.xlim(0,1)
plt.title("All")
plt.xlabel("Dice score" )
plt.ylabel("IC area in pixels")
plt.legend(p.legend_elements(),
                    loc="lower left", title="Classes")
#plt.show
dices = dice_flat_a + dice_flat_p + dice_flat_s
#dicess.append(flattened_s)
print((sum(dice_flat_a)/len(dice_flat_a)), (sum(dice_flat_p)/len(dice_flat_p)), (sum(dice_flat_s)/len(dice_flat_s)))
areas = area_flat_a + area_flat_p + area_flat_s
no_names = len(dice_flat_a)
adrians =  ["Adrian"]* no_names
pauls = ["Paul"]*no_names
sachas = ["Sacha"]*no_names
observers = adrians + pauls + sachas


total_dataframe = pd.DataFrame(dices, columns= ["Dice"])
total_dataframe["Areas"] = areas
total_dataframe["Observers"] = observers

import seaborn as sns
sns.set_theme()

# Load the penguins dataset
penguins = sns.load_dataset("penguins")

# Plot sepal width as a function of sepal_length across days
g = sns.lmplot(
    data = total_dataframe,
    x="Dice", y="Areas", hue = "Observers"
)

# Use more informative axis labels than are provided by default
g.set_axis_labels("Dice score", "Area (pixels)")


In [ ]:
#plt.boxplot(flattened_a)
data = [adrian_patient_avg, sacha_patient_avg, paul_patient_avg]

df = pd.DataFrame(adrian_patient_avg, columns = ['Adrian'])
df['Paul'] = paul_patient_avg
df['Sacha'] = sacha_patient_avg
boxplot = df.boxplot() 

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks")


sns.pairplot(total_dataframe, hue="Observers")

In [ ]:

from utils import IdSkip4
#new_mask_paul,new_id_paul,new_mask_adrian,new_id_adrian,new_mask_sacha,new_id_sacha, new_mask_h, new_id_h = IdSkip4(ids_paul, ids_adrian, ids_sacha, ids_h, masks_paul, masks_adrian, masks_sacha, masks_h)

PLIC_new_mask_paul,PLIC_new_id_paul,PLIC_new_mask_adrian,PLIC_new_id_adrian,PLIC_new_mask_sacha,PLIC_new_id_sacha, PLIC_new_mask_h, PLIC_new_id_h = IdSkip4(PLIC_ids_paul, PLIC_ids_adrian, PLIC_ids_sacha, PLIC_ids_h, PLIC_masks_paul, PLIC_masks_adrian, PLIC_masks_sacha, PLIC_masks_h)


In [ ]:
from utils import UnpackSlices, remove_extra_ALIC
adrian_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_adrian.npz'
sacha_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_sacha.npz'
paul_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_paul.npz'
h_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_h.npz'

slices_adrian, ids_adrian = UnpackSlices(adrian_path)
slices_paul, ids_paul = UnpackSlices(paul_path)
slices_sacha, ids_sacha = UnpackSlices(sacha_path)
slices_h, ids_h = UnpackSlices(h_path)





slices_array_a = remove_extra_ALIC(mask_ids =ids_adrian, CR_slices = slices_adrian, sheetname = "adrian")
slices_array_b = remove_extra_ALIC(mask_ids = ids_paul, CR_slices = slices_paul, sheetname = "paul")
slices_array_c = remove_extra_ALIC(mask_ids = ids_sacha, CR_slices = slices_sacha, sheetname = "sacha")
slices_array_d = remove_extra_ALIC(mask_ids = ids_h, CR_slices = slices_h, sheetname = "h")

In [ ]:
for i in range (0, len(PLIC_new_mask_adrian)):
    patient = PLIC_new_id_adrian[i]
    index_a = ids_adrian.tolist().index(patient)
    index_p = ids_paul.tolist().index(patient)
    index_s = ids_sacha.tolist().index(patient)
    index_h = ids_h.tolist().index(patient)
    for slices in slices_array_a[index_a]:
        if slices in slices_array_d[index_h]:
            fig, (ax1) = plt.subplots(1, 1, figsize=(12,3))
            title = str(patient) + "slice " + str(slices)
            ax1.set_title(title)
          
            PLIC_new_mask_adrian[i] = PLIC_new_mask_adrian[i].astype(float)
            PLIC_new_mask_h[i] = PLIC_new_mask_h[i].astype(float)
            PLIC_new_mask_adrian[i][PLIC_new_mask_adrian[i]==0] = np.nan
            PLIC_new_mask_h[i][PLIC_new_mask_h[i]==0] = np.nan
            
            seg_a = PLIC_new_mask_adrian[i][slices] 
            seg_d =  PLIC_new_mask_h[i][slices]
            #seg_p = PLIC_new_mask_paul[i][slices]
            #seg_s = PLIC_new_mask_sacha[i][slices]
            #seg_h = PLIC_new_mask_h[i][slices]
            #seg2_err = np.zeros([512,512])   + s
            #seg1_err_flip = ImageOps.flip(seg1_err)
            
            ax1.imshow(seg_a,  cmap='spring', interpolation="none", alpha=0.5)
            ax1.imshow(seg_d ,  cmap='plasma', interpolation="none", alpha=0.5)
            #ax2.imshow(seg_p,  cmap='jet', interpolation="none", alpha=0.5)
            #ax3.imshow(seg_s,  cmap='jet', interpolation="none", alpha=0.5)
            #ax4.imshow(seg_h,  cmap='jet', interpolation="none", alpha=0.5)

In [ ]:
from utils import StapleGunPLIC
staple_array, staple_ids, staple_slices = StapleGunPLIC(slices__a =slices_array_a, slices__b = slices_array_b, slices__c = slices_array_c, slices__d = slices_array_d, p_masks_a= PLIC_new_mask_adrian, p_masks_b= PLIC_new_mask_paul, p_masks_c= PLIC_new_mask_sacha, p_masks_d = PLIC_new_mask_h, p_ids_a= PLIC_new_id_adrian, p_ids_b = PLIC_new_id_paul, p_ids_c= PLIC_new_id_sacha, p_ids_d = PLIC_new_id_h)
#staple_array, staple_ids, staple_slices = StapleGun4(masks_a= new_mask_adrian, masks_b= new_mask_paul, masks_c= new_mask_sacha, masks_d = new_mask_h, ids_a= new_id_adrian, ids_b = new_id_paul, ids_c= new_id_sacha, ids_d = new_id_h)

In [ ]:
dice_array_adrian = DiceCaster(staple_array, staple_ids, staple_slices, new_mask_adrian, new_id_adrian)
adrian_total_avg, adrian_patient_avg = AverageDice(dice_array_adrian)

dice_array_paul = DiceCaster(staple_array, staple_ids, staple_slices, new_mask_paul, new_id_paul)
paul_total_avg, paul_patient_avg = AverageDice(dice_array_paul)

dice_array_sacha = DiceCaster(staple_array, staple_ids, staple_slices, new_mask_sacha, new_id_sacha)
sacha_total_avg, sacha_patient_avg = AverageDice(dice_array_sacha)

print("avg adrian: ", adrian_total_avg, "avg paul: ", paul_total_avg, "avg sacha: ", sacha_total_avg)




In [ ]:
new_slice_adrian = GetSlicesArray(new_mask_adrian)
new_slice_paul = GetSlicesArray(new_mask_paul)
new_slice_sacha = GetSlicesArray(new_mask_sacha)
percent = []
intersections = []
total_intersection = 0
total_union = 0
for i in new_id_adrian:
    ind_a = new_id_adrian.tolist().index(i)
    ind_p= new_id_paul.tolist().index(i)
    ind_s = new_id_sacha.tolist().index(i)
    print(ind_a, ind_p, ind_s)
    intersection = set(new_slice_adrian[ind_a]) & set(new_slice_paul[ind_p]) & set(new_slice_sacha[ind_s])
    print(new_slice_adrian[ind_a], new_slice_paul[ind_p], new_slice_sacha[ind_s])
    print(intersection)
    intersections.append(intersection)
    #union = set(slices_a[i]).union(set(slices_p[i]),set(slices_s[i]) )
    union = set(new_slice_adrian[ind_a])|set(new_slice_paul[ind_p])|set(new_slice_sacha[ind_s])
    smooth = 0.001
    percentage = len(intersection)/(len(union)+smooth)
    print(percentage)
    percent.append(percentage)
    total_intersection += len(intersection)
    total_union += len(union)

total_percentage = total_intersection/total_union
print(total_percentage)

In [ ]:
from scipy.ndimage import label, generate_binary_structure, maximum
def ALICRemover(mask_array_ALIC ,CR_slice_array):
    mask_array = mask_array_ALIC.copy()
    s = generate_binary_structure(2,2)
    s2 = [[1,1],
          [1,1]]
    for i in range(0, len(mask_array)):
        
        slices = CR_slice_array[i]
        print(len(mask_array[i]), len(slices))
        print("slices: ", slices)
        for slic in slices:
            print("slice: ", slic)
            
            labeled_mask, num_labels = label(mask_array[i][slic], structure = s)
            labs = []
            sums = []
            for j in range (1, num_labels+1):
                
                sum = np.sum(labeled_mask[labeled_mask == j])
                labs.append(j)
                sums.append(sum)
            #print(sums)
            #print(labs)
            if len(labs) != 0:

                max = np.max(sums)
                ind = sums.index(max)
                max_label = labs[ind]

              
                
                labeled_mask[labeled_mask != max_label] =0 
                labeled_mask[labeled_mask != 0] =1
                print("replaced")
                mask_array[i][slic] = labeled_mask
    return mask_array




In [ ]:
#dummy_mask_adrian = new_mask_adrian.copy()
#dummy_mask_paul = new_mask_paul.copy()
#dummy_mask_sacha = new_mask_sacha.copy()
#dummy_mask_h = new_mask_h.copy()

path_a = "/Users/olivia/Documents/PhD/MISTIE/mask_data/CRdata.xlsx"
adrian_slices = NewRemoveCoronaRadiataSlices(masks_adrian, ids_adrian, path_a, sheetname='adrian')
paul_slices = NewRemoveCoronaRadiataSlices(masks_paul,ids_paul, path_a, sheetname='paul')
sacha_slices = NewRemoveCoronaRadiataSlices(masks_sacha,ids_sacha, path_a, sheetname='sacha')
h_slices = NewRemoveCoronaRadiataSlices(masks_h,ids_h, path_a, sheetname='h')

In [ ]:


path_a = "/Users/olivia/Documents/PhD/MISTIE/mask_data/CRdata.xlsx"
adrian_slices_o = NewRemoveCoronaRadiataSlices(masks_adrian_old, ids_adrian_old, path_a, sheetname='adrian')


PLIC_mask_adrian = ALICRemover(masks_adrian_old, adrian_slices_o)
"""
PLIC_mask_paul = ALICRemover(masks_paul, paul_slices)
PLIC_mask_sacha = ALICRemover(masks_sacha, sacha_slices)
PLIC_mask_h = ALICRemover(masks_h, h_slices)

"""

In [ ]:
def SaveData(inputs, ids):
    path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_h.npz'
    ids = np.array(ids)
    #print("final shape: ", inputs.shape,  ids.shape)
    np.savez(path, slices = inputs,  ids = ids) 
    print("Saved data") 
save = SaveData(h_slices, ids_h)

In [ ]:
adrian_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_adrian.npz'
sacha_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_sacha.npz'
paul_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_paul.npz'
h_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_h.npz'

slices_adrian, ids_adrian = UnpackSlices(adrian_path)
slices_paul, ids_paul = UnpackSlices(paul_path)
slices_sacha, ids_sacha = UnpackSlices(sacha_path)
slices_h, ids_h = UnpackSlices(h_path)

In [ ]:
print(slices_h)

In [ ]:
def remove_extra_ALIC( mask_ids, CR_slices, sheetname ):
    tidy_slices = CR_slices.copy()
    alic_path = "/Users/olivia/Documents/PhD/MISTIE/mask_data/ALIC_data.xlsx"
    alic_frame = pd.read_excel(alic_path, sheet_name= sheetname, usecols = ['id','slice_index' ])
    ids = alic_frame['id'].astype(str).tolist()
    slice_index = alic_frame['slice_index'].astype(str).tolist()
    for d in range(0, len(ids)):
        id = ids[d]
        if id in mask_ids:
                
            print(id)
            placehold = mask_ids.tolist().index(id)
            
            slices = CR_slices[placehold].copy()
            if len(slices) != 0:

                print("slices before: ", slices)
                index_array = []
                print("test: ", slice_index[d])
                
                for i in range(1, len(slice_index[d])-1):
                    
                    index_array.append(int(slice_index[d][i]))
                print("to del: ", index_array)
                for index in sorted(index_array, reverse=True):
                    del slices[index]
                print("slices after: ", slices)
                print("tidy before: ", tidy_slices[placehold])
                tidy_slices[placehold] = slices
                print("tidy after: ", tidy_slices[placehold])

    return tidy_slices
#tidy = remove_extra_ALIC( PLIC_mask_paul, ids_paul, paul_slices, sheetname = 'paul' )


In [ ]:


for i in range(0, len(ids_h)):
    if len(h_slices[i]) != 0:
        for s in h_slices[i]:
            fig, (ax1) = plt.subplots(1, 1, figsize=(12,3))
            title = ids_h[i]
            ax1.set_title(title)

            ax1.imshow(PLIC_mask_h[i][s])


In [ ]:
from utils import UnpackSlices
adrian_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_adrian.npz'
sacha_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_sacha.npz'
paul_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_paul.npz'
h_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/CR_slices_h.npz'

slices_adrian, ids_adrian = UnpackSlices(adrian_path)
slices_paul, ids_paul = UnpackSlices(paul_path)
slices_sacha, ids_sacha = UnpackSlices(sacha_path)
slices_h, ids_h = UnpackSlices(h_path)





slices_array_a = remove_extra_ALIC(mask_ids =ids_adrian, CR_slices = slices_adrian, sheetname = "adrian")
slices_array_b = remove_extra_ALIC(mask_ids = ids_paul, CR_slices = slices_paul, sheetname = "paul")
slices_array_c = remove_extra_ALIC(mask_ids = ids_sacha, CR_slices = slices_sacha, sheetname = "sacha")
slices_array_d = remove_extra_ALIC(mask_ids = ids_h, CR_slices = slices_h, sheetname = "h")

In [ ]:
def DiceCaster2(masks_b, ids_b, slices_b, slices_b_ids, masks_a, ids_a, slices_a, slices_a_ids):
    
    id_a = ids_a.tolist()
    dice_array =[]
    for i in range(0, len(masks_b)):
        dices_array =[]
        patient_id = ids_b[i]
        print(patient_id)
        place_holder = id_a.index(patient_id)
        slices_place_a = slices_a_ids.tolist().index(patient_id)
        slices_place_b = slices_b_ids.tolist().index(patient_id)

        print(slices_b[i])
        for j in slices_b[slices_place_b]:
            
            
            print(" j: ", j, slices_b[slices_place_b], slices_a[slices_place_a])
            if j in slices_a[slices_place_a]:
                print("true")
                
  
          

                
     
                masks_a[i][j][masks_a[i][j] != 0] =1
                masks_b[i][j][masks_b[i][j] !=0 ] =1

                dice_coeff = compute_dice_coefficient(masks_a[i][j], masks_b[i][j])
                print("dice: ", dice_coeff)
                dice_coeff = dice_coeff * 100
                dices_array.append(dice_coeff)
            else:
                print("false")
        dice_array.append(dices_array)   
    return dice_array

In [ ]:
 
path_a = "/Users/olivia/Documents/PhD/MISTIE/mask_data/CRdata.xlsx"
patient = '2047'

#index1 = ct_ids.tolist().index(patient)
#index2 = staple_ids.index(patient)

#dice_arr = DiceCaster2(new_mask_adrian, new_id_adrian, cr_slices_adrian, new_mask_sacha, new_id_sacha, sacha_slices)

dice_arr = DiceCaster2(PLIC_new_mask_adrian, PLIC_new_id_adrian, slices_array_a, ids_adrian, PLIC_new_mask_sacha, PLIC_new_id_sacha, slices_array_c, ids_sacha)
dice_arr2 = DiceCaster2(PLIC_new_mask_adrian, PLIC_new_id_adrian, slices_array_a,ids_adrian,PLIC_new_mask_paul, PLIC_new_id_paul, slices_array_b, ids_paul)
dice_arr3 = DiceCaster2(PLIC_new_mask_paul, PLIC_new_id_paul, slices_array_b,ids_paul,PLIC_new_mask_sacha, PLIC_new_id_sacha, slices_array_c, ids_sacha)
dice_arr4 = DiceCaster2(PLIC_new_mask_h, PLIC_new_id_h, slices_array_d,ids_h, PLIC_new_mask_adrian, PLIC_new_id_adrian, slices_array_a, ids_adrian)
dice_arr5 = DiceCaster2(PLIC_new_mask_h, PLIC_new_id_h, slices_array_d, ids_h, PLIC_new_mask_paul, PLIC_new_id_paul, slices_array_b, ids_paul)
dice_arr6 = DiceCaster2(PLIC_new_mask_h, PLIC_new_id_h, slices_array_d, ids_h, PLIC_new_mask_sacha, PLIC_new_id_sacha, slices_array_c, ids_sacha)


AP_df = pd.DataFrame(dice_arr2, columns = ["Dice 1", "Dice 2", "Dice 3", "Dice 4"],index = PLIC_new_id_adrian)
AS_df = pd.DataFrame(dice_arr, columns = ["Dice 1", "Dice 2", "Dice 3", "Dice 4"], index = PLIC_new_id_adrian)
PS_df = pd.DataFrame(dice_arr3, columns = ["Dice 1", "Dice 2", "Dice 3", "Dice 4"], index = PLIC_new_id_paul)
HA_df = pd.DataFrame(dice_arr4, columns = ["Dice 1", "Dice 2", "Dice 3", "Dice 4"], index = PLIC_new_id_adrian)
HP_df = pd.DataFrame(dice_arr5, columns = ["Dice 1", "Dice 2", "Dice 3", "Dice 4", "Dice 5"], index = PLIC_new_id_adrian)
HS_df = pd.DataFrame(dice_arr6, columns = ["Dice 1", "Dice 2", "Dice 3", "Dice 4"], index = PLIC_new_id_adrian)

AP_total_avg, AP_patient_avg = AverageDice(dice_arr2)
AS_total_avg, AS_patient_avg = AverageDice(dice_arr)
PS_total_avg, PS_patient_avg = AverageDice(dice_arr3)
HA_total_avg, HA_patient_avg = AverageDice(dice_arr4)
HP_total_avg, HP_patient_avg = AverageDice(dice_arr5)
HS_total_avg, HS_patient_avg = AverageDice(dice_arr6)

AP_df["Average Dice"] = AP_patient_avg
AS_df["Average Dice"] = AS_patient_avg
PS_df["Average Dice"] = PS_patient_avg
HA_df["Average Dice"] = HA_patient_avg
HP_df["Average Dice"] = HP_patient_avg
HS_df["Average Dice"] = HS_patient_avg

total_df = pd.DataFrame([AP_total_avg, AS_total_avg, PS_total_avg, HA_total_avg, HP_total_avg, HS_total_avg], index =[ "Adrian Paul average Dice", "Adrian Sacha average Dice", "Paul Sacha average Dice", "H Adrian avg Dice", "H Paul average Dice", "H Sacha average Dice"] )
with pd.ExcelWriter('/Users/olivia/Documents/PhD/MISTIE/mask_data/PLIC_pairwise.xlsx') as writer:  
    AP_df.to_excel(writer, sheet_name='Adrian  Paul')
    AS_df.to_excel(writer, sheet_name='Adrian Sacha')
    PS_df.to_excel(writer, sheet_name='Paul Sacha')
    HA_df.to_excel(writer, sheet_name="H Adrian")
    HP_df.to_excel(writer, sheet_name="H Paul")
    HS_df.to_excel(writer, sheet_name="H Sacha")
    total_df.to_excel(writer, sheet_name = 'total')
"""
print(np.unique(new_mask_adrian[index3][15]))
plt.imshow(ct_scans[index1][15], cmap = "gray")
strdt = (new_mask_adrian[index3][15])
print(np.unique(strdt))
strdt[strdt == 0] = 'NaN'
#new_mask_adrian[index3][15][new_mask_adrian[index3][15]==0] = np.nan
plt.imshow(new_mask_adrian[index3][15], alpha = 0.5)
"""

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid
'''
fig, axs = plt.subplots(2, 2)
axs[0, 0].plot(x, y)
axs[0, 0].set_title('Axis [0, 0]')
axs[0, 1].plot(x, y, 'tab:orange')
axs[0, 1].set_title('Axis [0, 1]')
axs[1, 0].plot(x, -y, 'tab:green')
axs[1, 0].set_title('Axis [1, 0]')
axs[1, 1].plot(x, -y, 'tab:red')
axs[1, 1].set_title('Axis [1, 1]')
'''
'''
fig,axs = plt.subplots(1, 5)

for i in range(0,77):
    for j in range(0,4):
        axs[0, j].imshow(staple_array[i][j])
    for ax in axs.flat:
        ax.set(xlabel='x-label', ylabel='y-label')

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()
'''
for i in range(0,77):
    if len(staple_array[i]) != 0:
        ncols = len(staple_array[i])
        fig = plt.figure(figsize=(10., 10.))
        grid = ImageGrid(fig, 111,  # similar to subplot(111)
                        nrows_ncols=(1, ncols),  # creates 2x2 grid of axes
                        axes_pad=0.5,  # pad between axes in inch.
                        )

        for ax, im in zip(grid, staple_array[i]):
            # Iterating over the grid returns the Axes.
            ax.imshow(im)

        plt.show()

In [ ]:
slices_a = GetSlicesArray(new_mask_adrian)
slices_p = GetSlicesArray(new_mask_paul)
slices_s = GetSlicesArray(new_mask_sacha)

In [ ]:
#slices_s = GetSlicesArray(new_mask_adrian)
for id in ids_h:
    if id == '2011':
        place_holder = ids_h.tolist().index(id)
        print(h_slices[place_holder])
        
        plt.imshow(masks_h[place_holder][5])
        area = np.sum(masks_h[place_holder][5])
        print(area)

In [ ]:
CT_matched_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/ct_matched_training.npz'
ct_matched_scan, ct_matched_ids = UnpackNpz(CT_matched_path)

In [ ]:
from PIL import ImageOps
import cv2
def ImageFlipperUD(image):
    image = np.flip(image, axis=0) 
    return image

"""
pathE = "/Users/olivia/Documents/PhD/MISTIE/mask_data/CRdata.xlsx"
patient = '2110'
placehold = ct_matched_ids.tolist().index(patient)
placehold2 = staple_ids.index(patient)
placehold3 = PLIC_new_id_adrian.tolist().index(patient)
placehold4 = PLIC_new_id_paul.tolist().index(patient)
placehold5 = PLIC_new_id_sacha.tolist().index(patient)
placehold6 = PLIC_new_id_h.tolist().index(patient)
imslices = staple_slices[placehold2]

#ct_scans[placehold] = ImageFlipper(ct_scans[placehold])
#new_mask_sacha[placehold3] = ImageFlipper(new_mask_sacha[placehold3])
#new_mask_adrian[placehold3] = ImageFlipperUD(new_mask_adrian[placehold3])
#slices = NewRemoveCoronaRadiataSlices(new_mask_sacha,new_id_sacha, pathE, sheetname ='sacha')
slices = GetSlices(new_mask_adrian[placehold3])
print("mask_len: ", len(new_mask_adrian[0]))
print(imslices)
window = 80
level = 40
#imslices= slices[placehold3]
vmin = (level/2) - window
vmax = (level/2) + window

index = 0
#print(len(ct_scans[placehold]))
ct_matched_scan[placehold][imslices[0]-1][ct_matched_scan[placehold][imslices[0]-1]>vmax] = vmax
ct_matched_scan[placehold][imslices[0]-1][ct_matched_scan[placehold][imslices[0]-1]<vmin] = vmin
image = ct_matched_scan[placehold][imslices[index]-1]


translation_matrix = np.float32([ [1,0,0], [0,1,40] ])
plt.imshow(new_mask_adrian[placehold3][imslices[0]])
img_translation = cv2.warpAffine(new_mask_adrian[placehold3][imslices[0]].astype(np.uint8), translation_matrix, (512, 512))
plt.imshow(img_translation)
#print("ct org: ", sitk.GetImageFromArray(ct_scans[placehold]).GetOrigin())
#plt.imshow(new_mask_adrian[placehold3][imslices[index]], alpha = 0.5)

fig, (ax1) = plt.subplots(1, 1, figsize=(12,3))
ax1.set_title("Rater 1")

seg1_err = np.zeros([512,512])  + img_translation #+ new_mask_sacha[placehold3][imslices[index]]
#seg2_err = np.zeros([512,512])   + s
#seg1_err_flip = ImageOps.flip(seg1_err)
ax1.imshow(np.clip(image, -350, 50), cmap='gray')
ax1.imshow(seg1_err,  cmap='jet', interpolation="none", alpha=0.5)
"""
patients = ['2110', '2463', '2086', '2027', '2141', '2058' ]
tranform_array = [np.float32([ [1,0,0], [0,1,167] ]), np.float32([ [1,0,0], [0,1,50] ]), np.float32([ [1,0,0], [0,1,42] ]), np.float32([ [1,0,0], [0,1,-217] ]),np.float32([ [1,0,0], [0,1,-210] ]), np.float32([ [1,0,0], [0,1,40] ])]

for j in range(0, len(patients)):

    place = ct_matched_ids.tolist().index(patients[j])
    place_slice= ids_adrian.tolist().index(patients[j])
    place_a = PLIC_new_id_adrian.tolist().index(patients[j])
    place_p = PLIC_new_id_paul.tolist().index(patients[j])
    place_s = PLIC_new_id_sacha.tolist().index(patients[j])
    place_h = PLIC_new_id_h.tolist().index(patients[j])
    #place_staple = staple_ids.index(patients[j])
    slices = slices_array_a[place_slice]
    for i in range(0, len(slices)):
        print(patients[j])
        print(place_a)
        print(tranform_array[j])


        fig, (ax1) = plt.subplots(1, 1, figsize=(12,3))
        title = patients[j]
        ax1.set_title(title)

        imag = ct_matched_scan[place][slices[i]]
        window = 80
        level = 40
   
        vmin = (level/2) - window
        vmax = (level/2) + window
        #ct_matched_scan[place][slices[i]][ct_matched_scan[place][slices[i]]>vmax] = vmax
        #ct_matched_scan[place][slices[i]][ct_matched_scan[place][slices[i]]<vmin] = vmin
        seg = np.zeros([512,512])  + PLIC_new_mask_adrian[place_a][slices[i]] #+ new_mask_sacha[placehold3][imslices[index]]
        #seg2_err = np.zeros([512,512])   + s
        #seg1_err_flip = ImageOps.flip(seg1_err)
        ax1.imshow(imag, cmap='gray')
        


       
        PLIC_new_mask_adrian[place_a][slices[i]] = cv2.warpAffine(PLIC_new_mask_adrian[place_a][slices[i]].astype(np.uint8), tranform_array[j], (512, 512))
        #PLIC_new_mask_adrian[place_a] = PLIC_new_mask_adrian[place_a].astype(float)
        #PLIC_new_mask_adrian[place_a][slices[i]][PLIC_new_mask_adrian[place_a][slices[i]] == 0] = np.nan

        seg2_ = np.zeros([512,512])  + PLIC_new_mask_adrian[place_a][slices[i]]
    
        ax1.imshow(seg2_,  cmap='jet', interpolation="none", alpha=0.5)
        
        #new_mask_paul[place_p][slices[i]] = cv2.warpAffine(new_mask_paul[place_p][slices[i]].astype(np.uint8), tranform_array[j], (512, 512))
        #new_mask_h[place_h][slices[i]] = cv2.warpAffine(new_mask_h[place_h][slices[i]].astype(np.uint8), tranform_array[j], (512, 512))
        #new_mask_sacha[place_s][slices[i]] = cv2.warpAffine(new_mask_sacha[place_s][slices[i]].astype(np.uint8), tranform_array[j], (512, 512))
        #staple_array[place_staple][i] = cv2.warpAffine(staple_array[place_staple][i].astype(np.uint8), tranform_array[j], (512, 512))
        




In [ ]:
def Save2DData(cts, ct_ids, masks, mask_ids, slices_arr, slices_ids, path):
    ct_2d_arr , mask_2d_arr, id_2d_arr, slice_2d_arr = [], [] ,[], []
    for i in range(len(ct_ids)):
        patient = ct_ids[i]
        place_slice = slices_ids.tolist().index(patient)
        place_mask = mask_ids.tolist().index(patient)
        slices = slices_arr[place_slice]
        masks[place_mask] = masks[place_mask].astype(float)
        for slic in slices:
            window = 80
            level = 40
    
            vmin = (level/2) - window
            vmax = (level/2) + window
            cts[i][slic][cts[i][slic]>vmax] = vmax
            cts[i][slic][cts[i][slic]<vmin] = vmin
            ct_2d = cts[i][slic]

            masks[place_mask][slic][masks[place_mask][slic] != 0] = 1
            masks[place_mask][slic][masks[place_mask][slic] == 0] = np.nan
            mask_2d = masks[place_mask][slic]
            id_2d = ct_ids[i]
            ct_2d_arr.append(ct_2d)
            mask_2d_arr.append(mask_2d)
            id_2d_arr.append(id_2d)
            slice_2d_arr.append(slic)

    np.savez(path, cts = ct_2d_arr, masks = mask_2d_arr,  ids = id_2d_arr, slices = slice_2d_arr) 
def Unpack2DNpz(path):
    
    data = np.load(path, allow_pickle=True)
    cts = data['cts']
    masks = data['masks']
    ids = data['ids']
    slices = data['slices']
    return cts, masks, ids, slices
#data = Save2DData(ct_matched_scan,ct_matched_ids, PLIC_new_mask_adrian, PLIC_new_id_adrian, slices_array_a, ids_adrian, path =  '/Users/olivia/Documents/PhD/MISTIE/mask_data/adrian_matched_2d_training.npz')
cts, masks, ids, slices = Unpack2DNpz('/Users/olivia/Documents/PhD/MISTIE/mask_data/adrian_matched_2d_training.npz')
for i in range(len(cts)):

    fig, (ax1) = plt.subplots(1, 1, figsize=(12,3))
    title = str(ids[i]) + " " + str(slices[i])
    ax1.set_title(title)
    ax1.imshow(cts[i], cmap = 'gray')
    ax1.imshow(masks[i], cmap = 'jet', alpha = 0.5)
#data = Save2DData(ct_matched_scan,ct_matched_ids, PLIC_new_mask_adrian, PLIC_new_id_adrian, slices_array_a, ids_adrian, path =  '/Users/olivia/Documents/PhD/MISTIE/mask_data/adrian_matched_2d_training.npz')

In [ ]:
# get testing data not in matched CTs 

In [ ]:
fucked_up_array = []
for id in ct_ids:
    if id != '2121' and id != '2381':
        if id in new_id_adrian:

            place_holder = ct_ids.tolist().index(id)
            place_holder1 = new_id_adrian.tolist().index(id)
            print(id, " : ", len(ct_scans[place_holder]), len(new_mask_adrian[place_holder1]), len(new_mask_sacha[place_holder1]), len(new_mask_paul[place_holder1]), len(new_mask_h[place_holder1]))
            if len(ct_scans[place_holder]) != len(new_mask_adrian[place_holder1]):
                fucked_up_array.append(id)

print(fucked_up_array)

In [ ]:
from pathlib import Path
import nibabel as nib
from scipy import ndimage as ndi
def ReadMask(filedir, trans_affine):
    #seg = sitk.ReadImage(filedir, imageIO= "NiftiImageIO")  #="NiftiImageIO")
    seg = nib.load(filedir)
    print(seg.affine)
    seg_data = seg.get_fdata()
    print(seg_data.shape)
    seg_datat = ndi.affine_transform(seg_data, trans_affine)
    import matplotlib.pyplot as plt
    """
    def show_slices(slices):
        #Function to display row of image slices 
        fig, axes = plt.subplots(1, len(slices))
        for i, slice in enumerate(slices):
            axes[i].imshow(slice.T, cmap="gray", origin="lower")
        ax1.imshow(seg1_err,  cmap='jet', interpolation="none", alpha=0.5)
    slice_2 = seg_datat[:, :, 16, 0]
    show_slices([slice_2])
    """


    '''
    use this
    fig, (ax1) = plt.subplots(1, 1, figsize=(12,3))
    ax1.set_title("Rater 1")
    seg1_err = np.zeros([512,512])  + seg_datat[:, :, 16, 0]

    ax1.imshow(seg1_err,  cmap='jet', interpolation="none", alpha=0.5)
  
    '''

    """
    #print(seg.GetOrigin())
    #seg1_err = np.zeros([512,512]) + seg[20] 
    #seg_arr = sitk.GetArrayFromImage(seg)
    seg2_err = np.zeros([512,512]) + seg[15]
    seg_arr = seg_arr.astype(np.int64)
    seg3_err = np.zeros([512,512]) + seg[15]

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,3))

    # for display purposes, we will use "clip" to window the CT, as well as rotate the scan
   
    ax1.set_title("Rater 2")
 
    ax1.imshow(seg2_err,  cmap='jet', interpolation="none", alpha=0.5)

    # display segmentations from our three raters
    ax2.set_title("Rater 1")

    ax2.imshow(seg2_err,  cmap='jet', interpolation="none", alpha=0.5)
    ax3.set_title("Rater 2")

    ax3.imshow(seg3_err,  cmap='jet', interpolation="none", alpha=0.5)
    """
    return seg
patient = '2141'
id = ids_adrian.tolist().index(patient)
al = GetSlices(masks_adrian[id])
print(al)


ct_nib = nib.load("/Users/olivia/Documents/PhD/MISTIE/matched_scan/2141/Diagnostic CT/nifti.nii.gz")
ct_data = ct_nib.get_fdata()
print(ct_data.shape)
print("ct affn: ",ct_nib.affine)
"""
ct_vox_center = (np.array(ct_data.shape) - 1) / 2
from nibabel.affines import apply_affine
import numpy.linalg as npl
apply_affine(ct_nib.affine, ct_vox_center)
ct_vox2mask_vox = npl.inv(mask.affine).dot(ct_nib.affine)
affn = apply_affine(ct_vox2mask_vox, ct_vox_center)
print(affn)
"""
translation_affine = np.array([[1, 0, 0, 0],
                    [0, 1, 0, -199],
                    [0, 0, 1, -510.5],
                    [0, 0, 0, 1]])
identity = np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])
mask = ReadMask("/Users/olivia/Documents/PhD/MISTIE/adrianexp/2141/scan.hdr", translation_affine)
mask_2 = ReadMask("/Users/olivia/Documents/PhD/MISTIE/adrianexp/2141/scan.hdr",identity )
mask_2_data = mask_2.get_fdata()



import cv2
import numpy as np


num_rows, num_cols = mask_2.shape[:2]

translation_matrix = np.float32([ [1,0,-119], [0,1,-106] ])
img_translation = cv2.warpAffine(mask_2_data[:, :, 16, 0], translation_matrix, (512, 512))
#cv2.imshow('Translation', img_translation)
plt.imshow(img_translation)
plt.imshow(mask_2_data[:, :, 16, 0], alpha = 0.5)


transform_ct = ndi.affine_transform(ct_data, translation_affine)
#adj_ct = apply_affine(translation_affine, ct)
#ct_data = transform_ct.get_fdata()
import matplotlib.pyplot as plt
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")

slice_0 = transform_ct[26, :, :]
slice_1 = transform_ct[:, 30, :]
slice_2 = transform_ct[:, :, 16]
show_slices([slice_0, slice_1, slice_2])
plt.suptitle("Center slices for EPI image")  
#ct = ReadMask2("/Users/olivia/Documents/PhD/MISTIE/matched_scan/2027/Diagnostic CT/nifti.nii.gz")
def GetFiles(filedir):
    entries = Path(filedir)
    fname, id, pathlist = [],[],[]
    idTick = 0
 
    for entry in entries.iterdir():
     idTick += 1
     skip = np.array([2006, 2031, 2170])
     if entry.name[0:4] != "2006" and entry.name[0:4] != ".DS_" and entry.name[0:4] != "2031" and entry.name[0:4] != "2170" and entry.name[0:4] != "2035":
        fname.append(entry.name)
        id.append(idTick)
        pathlist.append(str(filedir) + "/" + str(entry.name))
    return np.array(fname), np.array(id), np.array(pathlist)
    


In [ ]:
from utils import ImageFlipper
patient = '2058'
placehold = ct_matched_ids.tolist().index(patient)
placehold2 = staple_ids.index(patient)
#ct_matched_scan[placehold] = ImageFlipper(ct_matched_scan[placehold])
slices = staple_slices[placehold2]
window = 80
level = 40

vmin = (level/2) - window
vmax = (level/2) + window
ct_matched_scan[placehold][slices[0]][ct_matched_scan[placehold][slices[0]]>vmax] = vmax
ct_matched_scan[placehold][slices[0]][ct_matched_scan[placehold][slices[0]]<vmin] = vmin
plt.imshow(ct_matched_scan[placehold][slices[0]], cmap = 'gray')

In [ ]:
def SaveData(inputs, ids):
    path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/ct_matched_training.npz'
    ids = np.array(ids)
    #print("final shape: ", inputs.shape,  ids.shape)
    np.savez(path, masks = inputs,  ids = ids) 
    print("Saved data") 
data = SaveData(ct_matched_scan, ct_matched_ids)

In [ ]:

for id in ct_matched_ids:
    if id != '2121' and id != '2085':
        print(id)
        id0 = ct_matched_ids.tolist().index(id)
        id1 = staple_ids.index(id)
        id2 = new_id_adrian.tolist().index(id)
        id3 = new_id_paul.tolist().index(id)
        id4 = new_id_sacha.tolist().index(id)
        id5 = new_id_h.tolist().index(id)
        imslices = staple_slices[id1]
        for i in range(0, len(imslices)):
            staple_array[id1][i] = staple_array[id1][i].astype(float)
            print(staple_array[id1][i].dtype)
            staple_array[id1][i][staple_array[id1][i] == 0] = np.nan
            blrag = staple_array[id1][i]
            seg1_err = np.zeros([512,512]) + staple_array[id1][i] + new_mask_adrian[id2][imslices[i]]
            seg2_err = np.zeros([512,512]) + staple_array[id1][i] + new_mask_paul[id3][imslices[i]]
            seg3_err = np.zeros([512,512]) + staple_array[id1][i] + new_mask_sacha[id4][imslices[i]]
            seg4_err = np.zeros([512,512]) + staple_array[id1][i]+ new_mask_h[id5][imslices[i]]
           
            

            window = 80
            level = 40
        
            vmin = (level/2) - window
            vmax = (level/2) + window
            ct_matched_scan[id0][imslices[i]][ct_matched_scan[id0][imslices[i]]>vmax] = vmax
            ct_matched_scan[id0][imslices[i]][ct_matched_scan[id0][imslices[i]]<vmin] = vmin
            image = ct_matched_scan[id0][imslices[i]]
            cttt = sitk.GetImageFromArray(image)
            # plot
            im = sitk.GetImageFromArray(new_mask_h[id5][imslices[i]])

            
            print("dir: ", im.GetDirection(), cttt.GetDirection() )
            print("org: ", im.GetOrigin(), cttt.GetOrigin())
            fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(12,3))

            # for display purposes, we will use "clip" to window the CT, as well as rotate the scan
            title = str(id) + " : " + str(i)
            ax1.set_title(title)
            ax1.imshow(np.clip(image, -350, 450), cmap='gray')

            # display segmentations from our three raters
            ax2.set_title("Rater 1")
            ax2.imshow(np.clip(image, -350, 50), cmap='gray')
            ax2.imshow(seg1_err,  cmap='jet', interpolation="none", alpha=0.7)
            ax3.set_title("Rater 2")
            ax3.imshow(np.clip(image, -350, 50), cmap='gray')
            ax3.imshow(seg2_err,  cmap='jet', interpolation="none", alpha=0.7)
            ax4.set_title("Rater 3")
            #ax4.imshow(np.rot90(np.clip(image, -350, 50), k=1), cmap='gray')
            ax4.imshow(np.clip(image, -350, 50), cmap='gray')
            ax4.imshow(seg3_err,  cmap='jet', interpolation="none", alpha=0.7)
            ax5.set_title("Rater 4")
            #ax4.imshow(np.rot90(np.clip(image, -350, 50), k=1), cmap='gray')
            ax5.imshow(np.clip(image, -350, 50), cmap='gray')
            ax5.imshow(seg4_err,  cmap='jet', interpolation="none", alpha=0.7)
            #plt.savefig("STAPLE_Error.png")
            plt.show()


In [ ]:

fig=plt.figure(figsize=(12, 90))
ax = []
rows = 100
columns = 4
#figure with 5 columns

for j in range(0, len(masks_h)):
  for i in h_slices[j]: #cr_slices_adrian[j]:
    ax.append(fig.add_subplot(rows, columns, i+1))
    title = str(ids_h[j])
    plt.title(title)
    plt.imshow(masks_h[j][i])
    
    
  
    plt.show()
    
  
